In [493]:
import numpy as np
import pandas as pd
import hvplot.pandas  # noqa
#For Monte Carlo
import random
#Visualization
import holoviews as hv
from tqdm import tqdm
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [460]:
!pip install yfinance


In [378]:
#load in ticker data for one portfolio only
stocks = ['AAPL','UNH','NEE','XOM','AMZN']
stock_list=yf.download(stocks, start="2010-01-01", end = "2021-01-01")


[*********************100%***********************]  5 of 5 completed


In [379]:
stock_list

Adj Close                                                \
                  AAPL        AMZN        NEE         UNH        XOM   
Date                                                                   
2010-01-04    6.515212    6.695000   9.108548   25.983183  41.998306   
2010-01-05    6.526477    6.734500   9.019599   25.941978  42.162289   
2010-01-06    6.422666    6.612500   9.108548   26.197441  42.526703   
2010-01-07    6.410791    6.500000   9.082892   27.202816  42.393082   
2010-01-08    6.453413    6.676000   9.004209   26.947355  42.223022   
...                ...         ...        ...         ...        ...   
2020-12-24  130.404572  158.634506  71.997063  331.815399  37.694801   
2020-12-28  135.068588  164.197998  72.486755  336.839508  37.821659   
2020-12-29  133.270157  166.100006  72.333122  338.202606  37.395775   
2020-12-30  132.133820  164.292496  72.890068  335.904755  37.694801   
2020-12-31  131.116043  162.846497  74.080727  341.444916  37.350471   

                 Close                                                ...  \
                  AAPL        AMZN        NEE         UNH        XOM  ...   
Date                                                                  ...   
2010-01-04    7.643214    6.695000  13.312500   31.530001  69.150002  ...   
2010-01-05    7.656429    6.734500  13.182500   31.480000  69.419998  ...   
2010-01-06    7.534643    6.612500  13.312500   31.790001  70.019997  ...   
2010-01-07    7.520714    6.500000  13.275000   33.009998  69.800003  ...   
2010-01-08    7.570714    6.676000  13.160000   32.700001  69.519997  ...   
...                ...         ...        ...         ...        ...  ...   
2020-12-24  131.970001  158.634506  74.980003  340.790009  41.599998  ...   
2020-12-28  136.690002  164.197998  75.489998  345.950012  41.740002  ...   
2020-12-29  134.869995  166.100006  75.330002  347.350006  41.270000  ...   
2020-12-30  133.720001  164.292496  75.910004  344.989990  41.599998  ...   
2020-12-31  132.690002  162.846497  77.150002  350.679993  41.220001  ...   

                  Open                                                \
                  AAPL        AMZN        NEE         UNH        XOM   
Date                                                                   
2010-01-04    7.622500    6.812500  13.260000   31.379999  68.720001   
2010-01-05    7.664286    6.671500  13.325000   31.730000  69.190002   
2010-01-06    7.656429    6.730000  13.160000   31.600000  69.449997   
2010-01-07    7.562500    6.600500  13.297500   31.590000  69.900002   
2010-01-08    7.510714    6.528000  13.245000   32.860001  69.690002   
...                ...         ...        ...         ...        ...   
2020-12-24  131.320007  159.695007  74.449997  336.440002  41.650002   
2020-12-28  133.990005  159.699997  75.430000  343.209991  41.689999   
2020-12-29  138.050003  165.496994  75.610001  347.989990  42.040001   
2020-12-30  135.580002  167.050003  75.629997  347.079987  41.330002   
2020-12-31  134.080002  163.750000  75.900002  345.040009  41.470001   

               Volume                                           
                 AAPL       AMZN       NEE       UNH       XOM  
Date                                                            
2010-01-04  493729600  151998000  10408800  12199500  27809100  
2010-01-05  601904800  177038000  11186400  11180700  30174700  
2010-01-06  552160000  143576000  13493200   9761100  35044700  
2010-01-07  477131200  220604000   9768800  11789800  27192100  
2010-01-08  447610800  196610000   9074400   7228700  24891800  
...               ...        ...       ...       ...       ...  
2020-12-24   54930100   29038000   1387500   1360600   8039000  
2020-12-28  124486200  113736000   3753700   2308200  23877500  
2020-12-29  121047300   97458000   3774900   2275700  20287700  
2020-12-30   96452100   64186000   3804800   1866000  23807300  
2020-12-31   99116600   59144000   4119400   1969000  22786500  

[2769

In [380]:
#Prices Dataframe
prices = stock_list["Adj Close"].dropna()


prices

,AAPL,AMZN,NEE,UNH,XOM
Date,,,,,
2010-01-04,6.515212,6.695000,9.108548,25.983183,41.998306
2010-01-05,6.526477,6.734500,9.019599,25.941978,42.162289
2010-01-06,6.422666,6.612500,9.108548,26.197441,42.526703
2010-01-07,6.410791,6.500000,9.082892,27.202816,42.393082
2010-01-08,6.453413,6.676000,9.004209,26.947355,42.223022
...,...,...,...,...,...
2020-12-24,130.404572,158.634506,71.997063,331.815399,37.694801
2020-12-28,135.068588,164.197998,72.486755,336.839508,37.821659
2020-12-29,133.270157,166.100006,72.333122,338.202606,37.395775


In [381]:
volume = stock_list["Volume"].dropna()



volume

,AAPL,AMZN,NEE,UNH,XOM
Date,,,,,
2010-01-04,493729600,151998000,10408800,12199500,27809100
2010-01-05,601904800,177038000,11186400,11180700,30174700
2010-01-06,552160000,143576000,13493200,9761100,35044700
2010-01-07,477131200,220604000,9768800,11789800,27192100
2010-01-08,447610800,196610000,9074400,7228700,24891800
...,...,...,...,...,...
2020-12-24,54930100,29038000,1387500,1360600,8039000
2020-12-28,124486200,113736000,3753700,2308200,23877500
2020-12-29,121047300,97458000,3774900,2275700,20287700


In [382]:
#returns df
returns = prices.pct_change().dropna()
returns.head()

,AAPL,AMZN,NEE,UNH,XOM
Date,,,,,
2010-01-05,0.001729,0.005900,-0.009765,-0.001586,0.003905
2010-01-06,-0.015906,-0.018116,0.009862,0.009847,0.008643
2010-01-07,-0.001849,-0.017013,-0.002817,0.038377,-0.003142
2010-01-08,0.006648,0.027077,-0.008663,-0.009391,-0.004011
2010-01-11,-0.008822,-0.024041,-0.000760,0.006727,0.011220


In [383]:
#covariance dataframe
cov = returns.cov()
cov.head()

,AAPL,AMZN,NEE,UNH,XOM
AAPL,0.000318,0.000160,0.000068,0.000121,0.000101
AMZN,0.000160,0.000400,0.000058,0.000112,0.000083
NEE,0.000068,0.000058,0.000156,0.000082,0.000076
UNH,0.000121,0.000112,0.000082,0.000271,0.000113
XOM,0.000101,0.000083,0.000076,0.000113,0.000225


In [384]:
#weights formation
np.random.seed(10) 
weights = np.random.random(len(stocks))
weights /= np.sum(weights)
weights

array([0.28855649, 0.00776345, 0.23705227, 0.28013281, 0.18649499])

In [385]:
#portfolio returns
rp = (returns.mean()*252)@weights 
rp

0.22164178723675276

In [386]:
#portfolio variance
port_var = weights@(cov*252)@weights 
port_var

0.03405238592835795

In [387]:
#portfolio sharpe ratio
rf = 0.02 #risk-free rate
sharpe = (rp-rf)/np.sqrt(port_var)
sharpe

1.092714647592553

In [388]:
#portfolio metrics function
def portfolio_metrics(weights, index='Trial'):
    
    '''
    This function generates the relative performance metrics that will be reported and will be used
    to find the optimal weights.
    
    Parameters:
    weights: initialized weights or optimal weights for performance reporting
    
    '''   
    
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=[index])
    return df

In [389]:
np.random.seed(42)
#Empty Container
portfolios = pd.DataFrame(columns=[*stocks, "Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"])
#Loop
for i in range(10000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    portfolios.loc[i, stocks] = weights
    metrics = portfolio_metrics(weights,i)
    portfolios.loc[i, ["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]] = \
    metrics.loc[i,["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]]
    
portfolios

,AAPL,UNH,NEE,XOM,AMZN,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
0,0.133197,0.338101,0.260318,0.2129,0.055485,0.270032,0.036719,0.191622,1.304816
1,0.065285,0.024308,0.362501,0.251571,0.296334,0.177953,0.030799,0.175496,0.900037
2,0.009284,0.437468,0.375464,0.095773,0.08201,0.258274,0.038151,0.195324,1.219894
3,0.105673,0.175297,0.302353,0.248877,0.1678,0.226419,0.031163,0.176531,1.169311
4,0.327909,0.074759,0.156568,0.196343,0.244421,0.218366,0.035294,0.187867,1.055888
...,...,...,...,...,...,...,...,...,...
9995,0.159747,0.16502,0.295248,0.179993,0.199992,0.220425,0.030793,0.17548,1.142155
9996,0.089994,0.044999,0.253494,0.364183,0.24733,0.199208,0.033196,0.182199,0.983584
9997,0.186606,0.243678,0.197558,0.059794,0.312363,0.204758,0.033982,0.184342,1.002258
9998,0.246642,0.18477,0.25895,0.111618,0.19802,0.228342,0.032556,0.180432,1.154685


In [390]:
#optimal portfolio
weight_df=portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()]

In [391]:
weight_df

,AAPL,UNH,NEE,XOM,AMZN,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
7148,0.206473,0.227147,0.353459,0.210815,0.002105,0.273298,0.033722,0.183635,1.379354


In [392]:
Portfolio_Returns_AAPL=pd.DataFrame(returns[['AAPL']].mul(0.206473))
Portfolio_Returns_AAPL

,AAPL
Date,
2010-01-05,0.000357
2010-01-06,-0.003284
2010-01-07,-0.000382
2010-01-08,0.001373
2010-01-11,-0.001821
...,...
2020-12-24,0.001592
2020-12-28,0.007385
2020-12-29,-0.002749


In [393]:
Portfolio_volume_AAPL=pd.DataFrame(volume[['AAPL']].mul(0.206473))
Portfolio_volume_AAPL

,AAPL
Date,
2010-01-04,1.019418e+08
2010-01-05,1.242771e+08
2010-01-06,1.140061e+08
2010-01-07,9.851471e+07
2010-01-08,9.241954e+07
...,...
2020-12-24,1.134158e+07
2020-12-28,2.570304e+07
2020-12-29,2.499300e+07


In [394]:
Portfolio_Returns_UNH=pd.DataFrame(returns[['UNH']].mul(0.227147))
Portfolio_Returns_UNH

,UNH
Date,
2010-01-05,-0.000360
2010-01-06,0.002237
2010-01-07,0.008717
2010-01-08,-0.002133
2010-01-11,0.001528
...,...
2020-12-24,0.002153
2020-12-28,0.003439
2020-12-29,0.000919


In [395]:
Portfolio_volume_UNH=pd.DataFrame(volume[['UNH']].mul(0.227147))
Portfolio_volume_UNH

,UNH
Date,
2010-01-04,2.771080e+06
2010-01-05,2.539662e+06
2010-01-06,2.217205e+06
2010-01-07,2.678018e+06
2010-01-08,1.641978e+06
...,...
2020-12-24,3.090562e+05
2020-12-28,5.243007e+05
2020-12-29,5.169184e+05


In [396]:
Portfolio_Returns_NEE=pd.DataFrame(returns[['NEE']].mul(0.353459))
Portfolio_Returns_NEE

,NEE
Date,
2010-01-05,-0.003452
2010-01-06,0.003486
2010-01-07,-0.000996
2010-01-08,-0.003062
2010-01-11,-0.000269
...,...
2020-12-24,0.003091
2020-12-28,0.002404
2020-12-29,-0.000749


In [207]:
Portfolio_volume_NEE=pd.DataFrame(volume[['NEE']].mul(0.353459))
Portfolio_volume_NEE

,NEE
Date,
2010-01-04,3.679084e+06
2010-01-05,3.953934e+06
2010-01-06,4.769293e+06
2010-01-07,3.452870e+06
2010-01-08,3.207428e+06
...,...
2020-12-24,4.904244e+05
2020-12-28,1.326779e+06
2020-12-29,1.334272e+06


In [206]:
Portfolio_Returns_XOM=pd.DataFrame(returns[['XOM']].mul(0.210815))
Portfolio_Returns_XOM

,XOM
Date,
2010-01-05,0.000823
2010-01-06,0.001822
2010-01-07,-0.000662
2010-01-08,-0.000846
2010-01-11,0.002365
...,...
2020-12-24,-0.000858
2020-12-28,0.000709
2020-12-29,-0.002374


In [205]:
Portfolio_volume_XOM=pd.DataFrame(volume[['XOM']].mul(0.210815))
Portfolio_volume_XOM

,XOM
Date,
2010-01-04,5.862575e+06
2010-01-05,6.361279e+06
2010-01-06,7.387948e+06
2010-01-07,5.732503e+06
2010-01-08,5.247565e+06
...,...
2020-12-24,1.694742e+06
2020-12-28,5.033735e+06
2020-12-29,4.276951e+06


In [204]:
Portfolio_Returns_AMZN=pd.DataFrame(returns[['AMZN']].mul(0.002105))
Portfolio_Returns_AMZN

,AMZN
Date,
2010-01-05,0.000012
2010-01-06,-0.000038
2010-01-07,-0.000036
2010-01-08,0.000057
2010-01-11,-0.000051
...,...
2020-12-24,-0.000008
2020-12-28,0.000074
2020-12-29,0.000024


In [397]:
Portfolio_volume_AMZN=pd.DataFrame(volume[['AMZN']].mul(0.002105))
Portfolio_volume_AMZN

,AMZN
Date,
2010-01-04,319955.79
2010-01-05,372664.99
2010-01-06,302227.48
2010-01-07,464371.42
2010-01-08,413864.05
...,...
2020-12-24,61124.99
2020-12-28,239414.28
2020-12-29,205149.09


In [398]:
Portfolio_Returns_Ticker=pd.concat([Portfolio_Returns_AAPL,Portfolio_Returns_UNH,Portfolio_Returns_NEE,Portfolio_Returns_XOM,Portfolio_Returns_AMZN])
Portfolio_Returns_Ticker

,AAPL,UNH,NEE,XOM,AMZN
Date,,,,,
2010-01-05,0.000357,NaN,NaN,NaN,NaN
2010-01-06,-0.003284,NaN,NaN,NaN,NaN
2010-01-07,-0.000382,NaN,NaN,NaN,NaN
2010-01-08,0.001373,NaN,NaN,NaN,NaN
2010-01-11,-0.001821,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-24,NaN,NaN,NaN,NaN,-0.000008
2020-12-28,NaN,NaN,NaN,NaN,0.000074
2020-12-29,NaN,NaN,NaN,NaN,0.000024


In [399]:
Portfolio_Returns_Ticker_sum=pd.DataFrame(Portfolio_Returns_Ticker.sum(axis=1),columns=['Returns'])
Portfolio_Returns_Ticker_sum

,Returns
Date,
2010-01-05,0.000357
2010-01-06,-0.003284
2010-01-07,-0.000382
2010-01-08,0.001373
2010-01-11,-0.001821
...,...
2020-12-24,-0.000008
2020-12-28,0.000074
2020-12-29,0.000024


In [400]:
Portfolio_volume_Ticker=pd.concat([Portfolio_volume_AAPL,Portfolio_volume_UNH,Portfolio_volume_NEE,Portfolio_volume_XOM,Portfolio_volume_AMZN])
Portfolio_volume_Ticker

,AAPL,UNH,NEE,XOM,AMZN
Date,,,,,
2010-01-04,1.019418e+08,NaN,NaN,NaN,NaN
2010-01-05,1.242771e+08,NaN,NaN,NaN,NaN
2010-01-06,1.140061e+08,NaN,NaN,NaN,NaN
2010-01-07,9.851471e+07,NaN,NaN,NaN,NaN
2010-01-08,9.241954e+07,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-24,NaN,NaN,NaN,NaN,61124.99
2020-12-28,NaN,NaN,NaN,NaN,239414.28
2020-12-29,NaN,NaN,NaN,NaN,205149.09


In [401]:
Portfolio_volume_Ticker_sum=pd.DataFrame(Portfolio_volume_Ticker.sum(axis=1), columns=['Volume'])
Portfolio_volume_Ticker_sum

,Volume
Date,
2010-01-04,1.019418e+08
2010-01-05,1.242771e+08
2010-01-06,1.140061e+08
2010-01-07,9.851471e+07
2010-01-08,9.241954e+07
...,...
2020-12-24,6.112499e+04
2020-12-28,2.394143e+05
2020-12-29,2.051491e+05


In [402]:
Portolio_Ticker=pd.concat([Portfolio_volume_Ticker_sum,Portfolio_Returns_Ticker_sum],join='outer')
Portolio_Ticker

,Volume,Returns
Date,,
2010-01-04,1.019418e+08,NaN
2010-01-05,1.242771e+08,NaN
2010-01-06,1.140061e+08,NaN
2010-01-07,9.851471e+07,NaN
2010-01-08,9.241954e+07,NaN
...,...,...
2020-12-24,NaN,-0.000008
2020-12-28,NaN,0.000074
2020-12-29,NaN,0.000024


In [403]:
short_window = 4
long_window = 100

In [404]:
Portfolio_SMA_SHORT = pd.DataFrame(prices['AAPL'].rolling(window=short_window).mean())
Portfolio_SMA_SHORT['UNH'] = pd.DataFrame(prices['UNH'].rolling(window=short_window).mean())
Portfolio_SMA_SHORT['NEE'] = pd.DataFrame(prices['NEE'].rolling(window=short_window).mean())
Portfolio_SMA_SHORT['XOM'] = pd.DataFrame(prices['XOM'].rolling(window=short_window).mean())
Portfolio_SMA_SHORT['AMZN'] = pd.DataFrame(prices['AMZN'].rolling(window=short_window).mean())

In [405]:
Portfolio_SMA_SHORT.dropna()

,AAPL,UNH,NEE,XOM,AMZN
Date,,,,,
2010-01-07,6.468786,26.331355,9.079897,42.270095,6.635500
2010-01-08,6.453337,26.572398,9.053812,42.326274,6.630750
2010-01-11,6.420838,26.869064,9.048254,42.459893,6.576000
2010-01-12,6.396102,26.922627,8.965721,42.449264,6.514750
2010-01-13,6.396635,26.825798,8.897299,42.429523,6.503625
...,...,...,...,...,...
2020-12-24,129.208923,328.482994,71.636978,37.731045,159.633255
2020-12-28,131.298840,330.885529,71.898632,37.683474,160.605503
2020-12-29,132.037468,333.889290,72.047462,37.690269,162.049004


In [406]:
Portfolio_SMA_LONG = pd.DataFrame(prices['AAPL'].rolling(window=long_window).mean())
Portfolio_SMA_LONG['UNH'] = pd.DataFrame(prices['UNH'].rolling(window=long_window).mean())
Portfolio_SMA_LONG['NEE'] = pd.DataFrame(prices['NEE'].rolling(window=long_window).mean())
Portfolio_SMA_LONG['XOM'] = pd.DataFrame(prices['XOM'].rolling(window=long_window).mean())
Portfolio_SMA_LONG['AMZN'] = pd.DataFrame(prices['AMZN'].rolling(window=long_window).mean())

In [407]:
Portfolio_SMA_LONG.dropna()

,AAPL,UNH,NEE,XOM,AMZN
Date,,,,,
2010-05-26,6.897059,26.447601,8.481622,40.593527,6.468255
2010-05-27,6.909036,26.429611,8.476606,40.551808,6.464655
2010-05-28,6.921974,26.409971,8.472739,40.502295,6.460040
2010-06-01,6.937153,26.387033,8.466270,40.441691,6.455535
2010-06-02,6.953401,26.361713,8.462190,40.391778,6.453690
...,...,...,...,...,...
2020-12-24,116.686542,314.965396,69.731498,33.939500,159.735235
2020-12-28,116.953485,315.314727,69.781722,33.937247,159.774700
2020-12-29,117.164634,315.662327,69.832716,33.932558,159.823200


In [408]:
Portfolio_SMA_LONG_sum=pd.DataFrame(Portfolio_SMA_LONG.sum(axis=1),columns=['SMA_LONG'])
Portfolio_SMA_LONG_sum
#Portfolio_Returns_Ticker_sum=pd.DataFrame(Portfolio_Returns_Ticker.sum(axis=1),columns=['Returns'])

,SMA_LONG
Date,
2010-01-04,0.000000
2010-01-05,0.000000
2010-01-06,0.000000
2010-01-07,0.000000
2010-01-08,0.000000
...,...
2020-12-24,695.058171
2020-12-28,695.761881
2020-12-29,696.415433


In [468]:
Portfolio_SMA_SHORT_sum=pd.DataFrame(Portfolio_SMA_SHORT.sum(axis=1),columns=['SMA_SHORT'])
Portfolio_SMA_SHORT_sum

,SMA_SHORT
Date,
2010-01-04,0.000000
2010-01-05,0.000000
2010-01-06,0.000000
2010-01-07,90.785633
2010-01-08,91.036570
...,...
2020-12-24,726.693195
2020-12-28,732.371977
2020-12-29,737.713492


In [477]:
Portolio_Ticker=Portolio_Ticker.assign(SHORT_Returns=Portfolio_SMA_SHORT_sum['SMA_SHORT'])
Portolio_Ticker=Portolio_Ticker.assign(LONG_Returns=Portfolio_SMA_LONG_sum['SMA_LONG'])
Portolio_Ticker_Weekly=Portolio_Ticker.resample('M').last()
Portolio_Ticker_Weekly=Portolio_Ticker_Weekly.dropna()
Portolio_Ticker_Weekly

,Volume,Returns,SHORT_Returns,LONG_Returns
Date,,,,
2010-03-31,193685.26,-0.000012,90.174726,0.000000
2010-04-30,257378.35,-0.000069,90.800963,0.000000
2010-06-30,410083.47,0.000013,81.911177,87.876221
2010-08-31,177897.76,0.000018,86.151556,86.464296
2010-09-30,320212.60,-0.000026,93.869294,86.870359
...,...,...,...,...
2020-07-31,340399.55,0.000078,646.748714,568.240971
2020-08-31,176226.39,0.000030,699.213604,613.610444
2020-09-30,206125.81,0.000003,664.019979,637.604375


In [478]:
X = Portolio_Ticker_Weekly[['SHORT_Returns', 'LONG_Returns','Volume']]
X

,SHORT_Returns,LONG_Returns,Volume
Date,,,
2010-03-31,90.174726,0.000000,193685.26
2010-04-30,90.800963,0.000000,257378.35
2010-06-30,81.911177,87.876221,410083.47
2010-08-31,86.151556,86.464296,177897.76
2010-09-30,93.869294,86.870359,320212.60
...,...,...,...
2020-07-31,646.748714,568.240971,340399.55
2020-08-31,699.213604,613.610444,176226.39
2020-09-30,664.019979,637.604375,206125.81


In [479]:
y = Portolio_Ticker_Weekly['Returns']

In [480]:
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=3)

In [481]:
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [485]:
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
rf_mean=mean_squared_error(y_test, y_pred)
rf_r2=r2_score(y_test, y_pred)

In [489]:
display(rf_mean)
display(rf_r2)

3.7199170209314137e-09

-0.8807665505564652

In [486]:
knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
knn_mean=mean_squared_error(y_test, y_pred)
knn_r2=r2_score(y_test, y_pred)

In [491]:
display(knn_mean)
display(knn_r2)

3.604321243586499e-09

-0.8223220556409707

In [494]:
svr = SVR(kernel='linear', gamma=0.1)
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
svm_mean=mean_squared_error(y_test, y_pred)
svm_r2=r2_score(y_test, y_pred)

In [495]:
display(svm_mean)
display(svm_r2)

2.890425882155206e-09

-0.46138106990865735

In [ ]:
SVR is better model as both mean error and R2 values are optimal compared to Random Forest and KNN values.